In [ ]:
params = {
    'N': 1,
    'seed': 0,
    'density': 500,
    'h': 1.0,
    'l2_regularisation': 0.000025,
    'bias_mean_initialisation': False,
    'batch_normalization': False,
    
    'resnet_block_kwargs': {
        'filters': [32, 32],
        'kernel_size': [100, 100],
        'padding': 'same',
        'activation': 'softplus',
        'add_input': True
    },
    
    'final_block_kwargs': {
        'filters': [1],
        'kernel_size': [100],
        'padding': 'same',
        'activation': None,
        'add_input': False
    },
    
    'blocks_length': 4,
    
    'metrics': {
        'kinetic_energy': 'mean_absolute_error',
        'kinetic_energy_density': 'mean_absolute_error',
        'derivative': 'mean_absolute_error'
    },
    
    'loss': {
        'kinetic_energy': 'mean_squared_error',
        'kinetic_energy_density': 'mean_squared_error',
        'derivative': 'mean_squared_error'
    },
    
    'loss_weights': {
        'kinetic_energy': 1.0,
        'kinetic_energy_density': 1.0,
        'derivative': 1.0
    },
    
    'optimizer_kwargs': {
        'learning_rate': 'WarmupExponentialDecay',
        'beta_1': 0.9,
        'beta_2': 0.999,
        'epsilon': 1e-07,
        'amsgrad': False,
        'clipnorm': 100.0,

        'learning_rate_kwargs': {
            'initial_learning_rate': 0.0001,
            'final_learning_rate': 0.0000000001,
            'decay_rate': 0.9,
            'decay_steps': 2000, # batches

            'cold_steps': 43700, # batches
            'warmup_steps': 0, # batches
            'cold_factor': 1.0
        }
    },
    
    'fit_kwargs': {
        'batch_size': 100,
        'epochs': 20000,
        'verbose': 0,
        #validation_split: 0.1
        'validation_freq': 1000,
        'shuffle': True,
        'initial_epoch': 0
    }
}

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pathlib import Path
import os

# from https://github.com/hauser-group/odft_tools
from odft_tools.layers import (
    IntegrateLayer,
    Continuous1DConvV1,
    Continuous1DConvV2
)

from odft_tools.models import (
    ResNetContConv1DModel,
    ResNetConv1DModel,
    ResNetCheckModel
)

data_path = '../datasets/orbital_free_DFT/'

In [ ]:
with h5py.File(data_path + 'M=100_training_data.hdf5', 'r') as f:
    keys = f.keys()
    print(keys)
    # build a dict (dataset.value has been deprecated. Use dataset[()] instead.)
    data = {key:f[key][()] for key in keys}

x = np.linspace(0, 1, 500)
dx = x[1] - x[0]
N = 1
# density is wavefunction squared
n = np.sum(data['wavefunctions'][:, :, :N]**2, axis=-1)
# integrate using trapezoidal rule:
V = np.sum(0.5*(data['potential'][:, :-1]*n[:, :-1] 
                + data['potential'][:, 1:]*n[:, 1:])           
           * dx, axis=-1)
# kinetic energy is total energy minus potential energy
T = np.sum(data['energies'][:, :N], axis=-1) - V
# kinetic energy derivative
dT_dn = np.expand_dims(np.sum(data['energies'][:, :N], axis=-1)/N, axis=-1) - data['potential']
n = n.reshape((-1, 500, 1))

with h5py.File(data_path + 'test_data.hdf5', 'r') as f:
    keys = f.keys()
    print(keys)
    # build a dict (dataset.value has been deprecated. Use dataset[()] instead.)
    data_test = {key:f[key][()] for key in keys}
    
# density is wavefunction squared
n_test = np.sum(data_test['wavefunctions'][:, :, :N]**2, axis=-1)
# integrate using trapezoidal rule:
V_test = np.sum(0.5*(data_test['potential'][:, :-1]*n_test[:, :-1]
                + data_test['potential'][:, 1:]*n_test[:, 1:])
                * dx, axis=-1)
# kinetic energy is total energy minus potential energy
T_test = np.sum(data_test['energies'][:, :N], axis=-1) - V_test
# kinetic energy derivative
dT_dn_test = - data_test['potential'] + np.expand_dims(np.sum(data_test['energies'][:, :N], axis=-1)/N, axis=-1) 
n_test = n_test.reshape((-1, 500, 1))

In [ ]:
def ResNet_KineticEnergyDensityFunctional(params):
    kernel_regularizer = tf.keras.regularizers.l2(params['l2_regularisation']) if params.get('l2_regularisation', 0.0) > 0.0 else None
    bias_initializer = tf.constant_initializer(value=params['dataset']['targets_mean']['kinetic_energy']) if params.get('bias_mean_initialisation', False) else None

    density = tf.keras.layers.Input(shape=params['density'], name='density')
    value = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(density)
    
    def resnet_block(input, filters, kernel_size, padding=None, activation=None, add_input=True):
        value = input
        for layer in range(len(filters)):
            value = tf.keras.layers.Conv1D(filters=filters[layer], 
                                        kernel_size=kernel_size[layer], 
                                        padding=padding,
                                        use_bias=True,
                                        activation= activation if layer < len(filters)-1 else None,
                                        kernel_regularizer=kernel_regularizer)(value)

        if add_input:
            value = tf.keras.layers.Add()([value, input])
        
        if activation is not None:
            return tf.keras.layers.Activation(activation=activation)(value)
        else:
            return value

    for layer in range(params['blocks_length']):
        if layer < params['blocks_length'] - 1:
            value = resnet_block(value, **params['resnet_block_kwargs'])
        else:
            value = resnet_block(value, **params['final_block_kwargs'])
            
    kinetic_energy_density = tf.keras.layers.Lambda(lambda x: tf.reduce_sum(x, axis=-1), name='kinetic_energy_density')(value)
    kinetic_energy = IntegrateLayer(params['h'], name='kinetic_energy')(kinetic_energy_density)
    return tf.keras.Model(inputs={'density': density}, outputs={'kinetic_energy': kinetic_energy, 'kinetic_energy_density': kinetic_energy_density})

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices((n.astype(np.float32), {'T': T.astype(np.float32), 'dT_dn': dT_dn.astype(np.float32)})).batch(100).repeat(10)

In [ ]:
model = ResNetCheckModel(params)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              loss={'T': 'mse', 'dT_dn': 'mse'}, 
              loss_weights={'T': 0.2, 'dT_dn': 1.0}, # As recommended by Manuel: scale the loss in T by 0.2
              metrics={'T': ['mae'], 'dT_dn': ['mae']})
model.build(input_shape=(None, 500, 1))
# model.summary()
training_dataset = tf.data.Dataset.from_tensor_slices((n.astype(np.float32), {'T': T.astype(np.float32), 'dT_dn': dT_dn.astype(np.float32)})).batch(100).repeat(10)
model.fit(training_dataset, epochs=50, verbose=0, validation_data=(n_test, {'T': T_test, 'dT_dn': dT_dn_test}), validation_freq=10)